<a href="https://colab.research.google.com/github/surajonuhu/startuppage/blob/main/vit_esemble_resner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import timm
import wandb
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip "/content/drive/MyDrive/archive.zip" -d "/content"

Archive:  /content/drive/MyDrive/archive.zip
  inflating: /content/PlantDoc-Dataset/test/Apple Scab Leaf/Apple Scab Leaf (1).jpg  
  inflating: /content/PlantDoc-Dataset/test/Apple Scab Leaf/Apple Scab Leaf (10).jpg  
  inflating: /content/PlantDoc-Dataset/test/Apple Scab Leaf/Apple Scab Leaf (2).jpg  
  inflating: /content/PlantDoc-Dataset/test/Apple Scab Leaf/Apple Scab Leaf (3).jpg  
  inflating: /content/PlantDoc-Dataset/test/Apple Scab Leaf/Apple Scab Leaf (4).jpg  
  inflating: /content/PlantDoc-Dataset/test/Apple Scab Leaf/Apple Scab Leaf (5).jpg  
  inflating: /content/PlantDoc-Dataset/test/Apple Scab Leaf/Apple Scab Leaf (6).jpg  
  inflating: /content/PlantDoc-Dataset/test/Apple Scab Leaf/Apple Scab Leaf (7).jpg  
  inflating: /content/PlantDoc-Dataset/test/Apple Scab Leaf/Apple Scab Leaf (8).jpg  
  inflating: /content/PlantDoc-Dataset/test/Apple Scab Leaf/Apple Scab Leaf (9).jpg  
  inflating: /content/PlantDoc-Dataset/test/Apple leaf/Apple leaf (1).jpg  
  inflating: /cont

In [5]:

IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS_BASE = 10  # for base models
EPOCHS_ENSEMBLE = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA_DIR = "/content/PlantDoc-Dataset"  # expects dataset/train and dataset/val
MODEL_DIR = "checkpoints"
os.makedirs(MODEL_DIR, exist_ok=True)
wandb.login(key="7901d254978ef4af4b881a72b10eda5eb36a4bb3")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])

val_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])

train_ds = datasets.ImageFolder(f"{DATA_DIR}/train", transform=train_tfms)
val_ds   = datasets.ImageFolder(f"{DATA_DIR}/test", transform=val_tfms)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

NUM_CLASSES = len(train_ds.classes)
print("Classes:", train_ds.classes)


Classes: ['Apple Scab Leaf', 'Apple leaf', 'Apple rust leaf', 'Bell_pepper leaf', 'Bell_pepper leaf spot', 'Blueberry leaf', 'Cherry leaf', 'Corn Gray leaf spot', 'Corn leaf blight', 'Corn rust leaf', 'Peach leaf', 'Potato leaf early blight', 'Potato leaf late blight', 'Raspberry leaf', 'Soyabean leaf', 'Squash Powdery mildew leaf', 'Strawberry leaf', 'Tomato Early blight leaf', 'Tomato Septoria leaf spot', 'Tomato leaf', 'Tomato leaf bacterial spot', 'Tomato leaf late blight', 'Tomato leaf mosaic virus', 'Tomato leaf yellow virus', 'Tomato mold leaf', 'grape leaf', 'grape leaf black rot']


In [7]:
def train_model(model, train_loader, val_loader, criterion, optimizer, name, epochs):
    best_acc = 0
    for epoch in range(epochs):
        # Train
        model.train()
        running_loss, correct = 0.0, 0
        for images, labels in train_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels)

        train_acc = correct.double() / len(train_loader.dataset)
        train_loss = running_loss / len(train_loader.dataset)

        # Validate
        model.eval()
        val_loss, val_correct = 0.0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels)

        val_acc = val_correct.double() / len(val_loader.dataset)
        val_loss /= len(val_loader.dataset)

        # Log metrics to wandb
        wandb.log({
            f"{name}_train_loss": train_loss,
            f"{name}_train_acc": train_acc.item(),
            f"{name}_val_loss": val_loss,
            f"{name}_val_acc": val_acc.item(),
            "epoch": epoch+1
        })

        print(f"{name} Epoch [{epoch+1}/{epochs}] "
              f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} "
              f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

        # Save best
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), f"{MODEL_DIR}/{name}.pth")
    return model

In [8]:
def train_base_models():
    criterion = nn.CrossEntropyLoss()

    # ViT
    vit = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=NUM_CLASSES).to(DEVICE)
    opt = optim.Adam(vit.parameters(), lr=1e-4)
    train_model(vit, train_loader, val_loader, criterion, opt, "vit_model", EPOCHS_BASE)

    # EfficientNet
    eff = timm.create_model('efficientnet_b0', pretrained=True, num_classes=NUM_CLASSES).to(DEVICE)
    opt = optim.Adam(eff.parameters(), lr=1e-4)
    train_model(eff, train_loader, val_loader, criterion, opt, "effnet_model", EPOCHS_BASE)

    # ResNet50
    res = models.resnet50(pretrained=True)
    res.fc = nn.Linear(res.fc.in_features, NUM_CLASSES)
    res = res.to(DEVICE)
    opt = optim.Adam(res.parameters(), lr=1e-4)
    train_model(res, train_loader, val_loader, criterion, opt, "resnet_model", EPOCHS_BASE)

In [9]:
class EnsembleNet(nn.Module):
    def __init__(self, num_classes):
        super(EnsembleNet, self).__init__()

        # ViT
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=False, num_classes=num_classes)
        self.vit.load_state_dict(torch.load(f"{MODEL_DIR}/vit_model.pth"))
        self.vit.head = nn.Identity()
        vit_dim = self.vit.num_features

        # EfficientNet
        self.eff = timm.create_model('efficientnet_b0', pretrained=False, num_classes=num_classes)
        self.eff.load_state_dict(torch.load(f"{MODEL_DIR}/effnet_model.pth"))
        self.eff.classifier = nn.Identity()
        eff_dim = self.eff.num_features

        # ResNet
        self.res = models.resnet50(pretrained=False)
        self.res.fc = nn.Linear(self.res.fc.in_features, num_classes)
        self.res.load_state_dict(torch.load(f"{MODEL_DIR}/resnet_model.pth"))
        self.res.fc = nn.Identity()
        res_dim = 2048

        # Freeze
        for net in [self.vit, self.eff, self.res]:
            for p in net.parameters():
                p.requires_grad = False

        # Classifier
        concat_dim = vit_dim + eff_dim + res_dim
        self.fc = nn.Sequential(
            nn.ReLU(),
            nn.Linear(concat_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
            f1 = self.vit(x)
            f2 = self.eff(x)
            f3 = self.res(x)
            combined = torch.cat((f1, f2, f3), dim=1)
            return self.fc(combined)

In [10]:

def evaluate_model(model, loader, class_names):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    cm = confusion_matrix(y_true, y_pred)
    report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)

    # Log to wandb
    wandb.log({
        "confusion_matrix": wandb.plot.confusion_matrix(
            y_true=y_true,
            preds=y_pred,
            class_names=class_names
        ),
        "classification_report": report
    })

    print(classification_report(y_true, y_pred, target_names=class_names))


In [11]:
if __name__ == "__main__":
    wandb.init(project="plant-disease-ensemble", name="vit_eff_resnet_ensemble")

    # Train base models (only once, comment out later if already trained)
    train_base_models()

    # Ensemble training
    ensemble = EnsembleNet(NUM_CLASSES).to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(ensemble.fc.parameters(), lr=1e-4)

    train_model(ensemble, train_loader, val_loader, criterion, optimizer, "ensemble_model", EPOCHS_ENSEMBLE)

    # Evaluate ensemble
    evaluate_model(ensemble, val_loader, train_ds.classes)

    wandb.finish()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

vit_model Epoch [1/10] Train Loss: 2.0329 Acc: 0.3959 Val Loss: 1.2844 Acc: 0.6186
vit_model Epoch [2/10] Train Loss: 0.8873 Acc: 0.7094 Val Loss: 0.9823 Acc: 0.6314
vit_model Epoch [3/10] Train Loss: 0.5100 Acc: 0.8217 Val Loss: 1.2196 Acc: 0.6229
vit_model Epoch [4/10] Train Loss: 0.4678 Acc: 0.8394 Val Loss: 1.3312 Acc: 0.6017
vit_model Epoch [5/10] Train Loss: 0.2635 Acc: 0.9167 Val Loss: 1.4244 Acc: 0.6568
vit_model Epoch [6/10] Train Loss: 0.2569 Acc: 0.9154 Val Loss: 1.4822 Acc: 0.5805
vit_model Epoch [7/10] Train Loss: 0.2112 Acc: 0.9348 Val Loss: 1.6612 Acc: 0.5890
vit_model Epoch [8/10] Train Loss: 0.1586 Acc: 0.9478 Val Loss: 1.5646 Acc: 0.6229
vit_model Epoch [9/10] Train Loss: 0.1286 Acc: 0.9547 Val Loss: 1.7445 Acc: 0.5593
vit_model Epoch [10/10] Train Loss: 0.1815 Acc: 0.9426 Val Loss: 1.5874 Acc: 0.6568


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

effnet_model Epoch [1/10] Train Loss: 3.0709 Acc: 0.1904 Val Loss: 2.6455 Acc: 0.2754
effnet_model Epoch [2/10] Train Loss: 1.5502 Acc: 0.5445 Val Loss: 2.0619 Acc: 0.3856
effnet_model Epoch [3/10] Train Loss: 0.9809 Acc: 0.7137 Val Loss: 1.7944 Acc: 0.5042
effnet_model Epoch [4/10] Train Loss: 0.6222 Acc: 0.8329 Val Loss: 1.6683 Acc: 0.5339
effnet_model Epoch [5/10] Train Loss: 0.4436 Acc: 0.8851 Val Loss: 1.6243 Acc: 0.5508
effnet_model Epoch [6/10] Train Loss: 0.2952 Acc: 0.9318 Val Loss: 1.5604 Acc: 0.5763
effnet_model Epoch [7/10] Train Loss: 0.2244 Acc: 0.9503 Val Loss: 1.5865 Acc: 0.5932
effnet_model Epoch [8/10] Train Loss: 0.1849 Acc: 0.9607 Val Loss: 1.6126 Acc: 0.5847
effnet_model Epoch [9/10] Train Loss: 0.1402 Acc: 0.9750 Val Loss: 1.6083 Acc: 0.5890
effnet_model Epoch [10/10] Train Loss: 0.1154 Acc: 0.9780 Val Loss: 1.6211 Acc: 0.5975


...

100%|██████████| 97.8M/97.8M [00:00<00:00, 177MB/s]


resnet_model Epoch [1/10] Train Loss: 2.1617 Acc: 0.3873 Val Loss: 1.4645 Acc: 0.5297
resnet_model Epoch [2/10] Train Loss: 1.0959 Acc: 0.6662 Val Loss: 1.2051 Acc: 0.6102
resnet_model Epoch [3/10] Train Loss: 0.7308 Acc: 0.7781 Val Loss: 1.1638 Acc: 0.6102
resnet_model Epoch [4/10] Train Loss: 0.5069 Acc: 0.8415 Val Loss: 1.1571 Acc: 0.6398
resnet_model Epoch [5/10] Train Loss: 0.4005 Acc: 0.8735 Val Loss: 1.2775 Acc: 0.6271
resnet_model Epoch [6/10] Train Loss: 0.2906 Acc: 0.9184 Val Loss: 1.1989 Acc: 0.6186
resnet_model Epoch [7/10] Train Loss: 0.2162 Acc: 0.9391 Val Loss: 1.4072 Acc: 0.6314
resnet_model Epoch [8/10] Train Loss: 0.1941 Acc: 0.9456 Val Loss: 1.2033 Acc: 0.6483
resnet_model Epoch [9/10] Train Loss: 0.1682 Acc: 0.9521 Val Loss: 1.4844 Acc: 0.6314
resnet_model Epoch [10/10] Train Loss: 0.1695 Acc: 0.9525 Val Loss: 1.4741 Acc: 0.6186


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ensemble_model Epoch [1/10] Train Loss: 1.2943 Acc: 0.7535 Val Loss: 0.9994 Acc: 0.6864
ensemble_model Epoch [2/10] Train Loss: 0.2120 Acc: 0.9672 Val Loss: 0.9618 Acc: 0.6822
ensemble_model Epoch [3/10] Train Loss: 0.1367 Acc: 0.9698 Val Loss: 0.9734 Acc: 0.7034
ensemble_model Epoch [4/10] Train Loss: 0.1011 Acc: 0.9754 Val Loss: 1.0016 Acc: 0.6949
ensemble_model Epoch [5/10] Train Loss: 0.0765 Acc: 0.9827 Val Loss: 1.0217 Acc: 0.6992
ensemble_model Epoch [6/10] Train Loss: 0.0705 Acc: 0.9823 Val Loss: 1.0728 Acc: 0.7076
ensemble_model Epoch [7/10] Train Loss: 0.0598 Acc: 0.9845 Val Loss: 1.1458 Acc: 0.6864
ensemble_model Epoch [8/10] Train Loss: 0.0610 Acc: 0.9840 Val Loss: 1.1068 Acc: 0.6949
ensemble_model Epoch [9/10] Train Loss: 0.0558 Acc: 0.9814 Val Loss: 1.1589 Acc: 0.6907
ensemble_model Epoch [10/10] Train Loss: 0.0562 Acc: 0.9858 Val Loss: 1.2041 Acc: 0.6992
                            precision    recall  f1-score   support

           Apple Scab Leaf       0.70      0.70   

effnet_model_train_acc,▁▄▆▇▇█████
effnet_model_train_loss,█▄▃▂▂▁▁▁▁▁
effnet_model_val_acc,▁▃▆▇▇█████
effnet_model_val_loss,█▄▃▂▁▁▁▁▁▁
ensemble_model_train_acc,▁▇████████
ensemble_model_train_loss,█▂▁▁▁▁▁▁▁▁
ensemble_model_val_acc,▂▁▇▄▆█▂▄▃▆
ensemble_model_val_loss,▂▁▁▂▃▄▆▅▇█
epoch,▁▂▃▃▄▅▆▆▇█▁▂▃▃▄▅▆▆▇█▁▂▃▃▄▅▆▆▇█▁▂▃▃▄▅▆▆▇█
resnet_model_train_acc,▁▄▆▇▇█████
+7,...
